<a href="https://colab.research.google.com/github/gnodking7/WDAnepv/blob/main/Example3%264.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective
In this example, we use the UCI datasets Vehicle and LSVT to demonstrate the classification accuracy of WDA-nepv in comparison to those of WDA-gd [1] and WDA-eig [3].

We will go over the background of the example in the following order:
1.   What are Vehicle and LSVT datasets?
2.   What are WDA-nepv, WDA-gd, and WDA-eig?
3.   Example Setup

### What are Vehicle and LSVT datasets?
UCI Machine Learning Repository is a collection of datasets that are widely used by the machine learning community for the empirical analysis of machine learning algorithms. In particular, the repository provides a wide variety of datasets that are suitable for clustering and classification tasks. 

Among the datasets, we choose the real-life datasets named Vehicle and LSVT. Vehicle dataset consists of 2D silhouette images of four types of vehicle. LSVT is a dataset consisting of phonations of Parkinson's disease subjects who participated in a voice rehabilitation treatment. LSVT dataset is suitable for a binary class classification problem as the classes correspond to 'acceptable' and 'unacceptable' phonations. Vehicle dataset has 4 classes with a total number of 946 data points of dimension 18 and LSVT dataset has 2 classes with a total number of 126 data points of dimension 309.

In [4]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [5]:
!pip install pymanopt
!pip install autograd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 101 kB 3.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import WDA_datasets as datasets
import WDAnepv
import WDAgd
import WDAeig
import numpy as np
import matplotlib.pylab as pl

In [7]:
# Call KNN package
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import time

# Set parameters
reg = 0.01  # regularization parameter
e = 1 # perturbation parameter

### What are WDA-nepv, WDA-gd, and WDA-eig?
WDA-nepv, WDA-gd, and WDA-eig are algorithms that solves Wasserstein Discriminant Analysis (WDA) [1]:
$$\max_{\mathbf{P}^T\mathbf{P}=I_p}\frac{\mbox{tr}(\mathbf{P}^T\mathbf{C}_b(\mathbf{P})\mathbf{P})}{\mbox{tr}(\mathbf{P}^T\mathbf{C}_w(\mathbf{P})\mathbf{P})}$$
where $\mathbf{C}_b(\mathbf{P})$ and $\mathbf{C}_w(\mathbf{P})$
are defined as the between and within cross-covariance matrices
$$\mathbf{C}_b(\mathbf{P}_k) = \sum_{c,c'>c}  \sum_{ij}{T}_{ij}^{c,c'}(\mathbf{P}_k)(\mathbf{x}_i^c-\mathbf{x}_j^{c'})(\mathbf{x}_i^c-\mathbf{x}_j^{c'})^T$$
$$\mathbf{C}_w(\mathbf{P}_k) = \sum_c  \sum_{ij}{T}_{ij}^{c,c}(\mathbf{P}_k)(\mathbf{x}_i^c-\mathbf{x}_j^{c})(\mathbf{x}_i^c-\mathbf{x}_j^{c})^T $$
The matrices $\mathbf{T}^{c,c'}$ and $\mathbf{T}^{c,c}$ are transport matrices and can be computed using Acc_SK.

WDA-gd is a steepest-descent algorithm and incurs extra costs for computing the gradients. WDA-eig solves a nonlinear generalized eigenvalue problem associated with an approximate problem to WDA.

WDA-nepvs is gradient-free and makes no approximation to WDA. In a nutshell, WDA-nepv can be described as an algorithm that iteratively updates the projection matrix $\mathbf{P}_k$ by 
$$\mathbf{P}_{k+1} = \mbox{argmax}_{\mathbf{P}^T\mathbf{P}=I_p}
\frac{\mbox{tr}(\mathbf{P}^T\mathbf{C}_b(\mathbf{P}_k)\mathbf{P})}
{\mbox{tr}(\mathbf{P}^T\mathbf{C}_w(\mathbf{P}_k)\mathbf{P})}$$
which, as a standard trace ratio problem, can be solved by the Self-Consistent-Field (SCF) method [2].

### Example Setup
As a supervised linear dimensionality reduction method, WDA seeks an optimal projection such that the class structure of the projected data vectors becomes more pronounced. For this reason, the effectiveness of WDA is measured by the classification accuracy.

Following the standard practice in classification tasks, each dataset is divided into the training dataset and the testing dataset. In this example, we consider 50% training and 50% testing split. After the algorithm is trained on the training dataset and an optimal projection matrix $\mathbf{P}$ is obtained, the testing dataset is then projected onto a lower dimensional subspace using this projection $\mathbf{P}$. Then, the classification accuracy of the algorithm is measured by using K-Nearest-Neighbors classifier (KNN) on the projected testing dataset. 

Two different experiments are considered:


*   With fixed subspace dimension $p=5$, various KNN neighbors $K\in\{1,3,5,7,9,11,13,15,17,19\}$ are considered. 
*   With fixed KNN neighbors $K=11$, various subspace dimension $p\in\{1,2,3,4,5\}$ for Vehicle dataset and $p\in\{5,10,15,20,25\}$ for LSVT dataset are considered.

The regularization parameter $\lambda=0.01$ is fixed throughout and the stopping tolerance parameter is set at $10^{-5}$. Initial projection matrix $\mathbf{P}_0$ is randomly chosen. All classification results are averaged over 20 trials.

In [ ]:
# Vehicle dataset
n_trial = 20
# Dict of success rate
Acclist_gd, Acclist_eig, Acclist_nepv = dict(), dict(), dict()
# List of running time
Time_gd, Time_eig, Time_nepv = [], [], []

# Trials
for i in range(n_trial):
    print('Starting Trial #:', i)

    TR, TR_L, TST, TST_L = datasets.load_uci('vehicle')
    d = TR.shape[1]

    time_gd, time_eig, time_nepv = [], [], []

    # For various subspace dimension p
    for p in [1, 2, 3, 4, 5]:
        acclist_gd, acclist_eig, acclist_nepv = [], [], []
        x0 = np.random.randn(d, p)
        P0, r = np.linalg.qr(x0) # random initial projection

        start1 = time.time()
        Pgd, proj1, Itr1, PROJ1 = WDAgd.wda_gd(TR, TR_L, p, reg, P0)    # WDAgd
        end1 = time.time()
        time1 = end1 - start1
        time_gd.append(time1)

        start2 = time.time()
        Peig, proj2, WDA_Val2, PROJ2, Sub_Err2 = WDAeig.wda_eig(TR, TR_L, p, reg, P0, Breg=e)   # WDAeig
        end2 = time.time()
        time2 = end2 - start2
        time_eig.append(time2)

        start3 = time.time()
        Pnepv, proj3, WDA_Val3, PROJ3, Sub_Err3 = WDAnepv.wda_nepv(TR, TR_L, p, reg, P0, Breg=e, tol=1e-5)  # WDAnepv
        end3 = time.time()
        time3 = end3 - start3
        time_nepv.append(time3)

        # For various KNN neighbors
        for K in [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]:
            # KNN models
            model1 = KNeighborsClassifier(n_neighbors=K)
            model1.fit(proj1(TR), TR_L)
            model2 = KNeighborsClassifier(n_neighbors=K)
            model2.fit(proj2(TR), TR_L)
            model3 = KNeighborsClassifier(n_neighbors=K)
            model3.fit(proj3(TR), TR_L)

            # Test
            predicted1 = model1.predict(proj1(TST))
            acc1 = accuracy_score(TST_L, predicted1)
            predicted2 = model2.predict(proj2(TST))
            acc2 = accuracy_score(TST_L, predicted2)
            predicted3 = model3.predict(proj3(TST))
            acc3 = accuracy_score(TST_L, predicted3)

            # Store accuracy for each K
            acclist_gd.append(acc1)
            acclist_eig.append(acc2)
            acclist_nepv.append(acc3)

        # Store accuracy for each p
        if p not in Acclist_gd:
            Acclist_gd[p] = []
        if p not in Acclist_eig:
            Acclist_eig[p] = []
        if p not in Acclist_nepv:
            Acclist_nepv[p] = []
        Acclist_gd[p].append(acclist_gd)
        Acclist_eig[p].append(acclist_eig)
        Acclist_nepv[p].append(acclist_nepv)

    # Store running time    
    Time_gd.append(time_gd)
    Time_eig.append(time_eig)
    Time_nepv.append(time_nepv)

avg_gd, avg_eig, avg_nepv = [], [], []
for p in [1, 2, 3, 4, 5]:
    avg_gd.append(np.mean(Acclist_gd[p], 0))
    avg_eig.append(np.mean(Acclist_eig[p], 0))
    avg_nepv.append(np.mean(Acclist_nepv[p], 0))

In [ ]:
# Call KNN package
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import time

# Load UCI datasets
Veh_TR, Veh_TR_L, Veh_TST, Veh_TST_L = datasets.load_uci('vehicle')
LSVT_TR, LSVT_TR_L, LSVT_TST, LSVT_TST_L = datasets.load_uci('lsvt')

reg = 0.01  # regularization parameter
e = 1 # perturbation parameter

n_trial = 20
# Dict of success rate
Acclist_gd, Acclist_eig, Acclist_nepv = dict(), dict(), dict()
# List of running time
Time_gd, Time_eig, Time_nepv = [], [], []

# Trials for LSVT
d = LSVT_TR.shape[1] # dimension size
TR, TR_L, TST, TST_L = LSVT_TR, LSVT_TR_L, LSVT_TST, LSVT_TST_L
for i in range(n_trial):
    print(i)

    time_gd, time_eig, time_nepv = [], [], []

    # For various subspace dimension p
    for p in [5, 10, 15, 20, 25]:
        acclist_gd, acclist_eig, acclist_nepv = [], [], []
        x0 = np.random.randn(d, p)
        P0, r = np.linalg.qr(x0) # random initial projection

        start1 = time.time()
        Pgd, proj1, Itr1, PROJ1 = WDAgd.wda_gd(TR, TR_L, p, reg, P0)    # WDAgd
        end1 = time.time()
        time1 = end1 - start1
        time_gd.append(time1)

        start2 = time.time()
        Peig, proj2, WDA_Val2, PROJ2, Sub_Err2 = WDAeig.wda_eig(TR, TR_L, p, reg, P0, Breg=e)   # WDAeig
        end2 = time.time()
        time2 = end2 - start2
        time_eig.append(time2)

        start3 = time.time()
        Pnepv, proj3, WDA_Val3, PROJ3, Sub_Err3 = WDAnepv.wda_nepv(TR, TR_L, p, reg, P0, Breg=e, tol=1e-5)  # WDAnepv
        end3 = time.time()
        time3 = end3 - start3
        time_nepv.append(time3)

        # For various KNN neighbors
        for K in [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]:
            # KNN models
            model1 = KNeighborsClassifier(n_neighbors=K)
            model1.fit(proj1(TR), TR_L)
            model2 = KNeighborsClassifier(n_neighbors=K)
            model2.fit(proj2(TR), TR_L)
            model3 = KNeighborsClassifier(n_neighbors=K)
            model3.fit(proj3(TR), TR_L)

            # Test
            predicted1 = model1.predict(proj1(TST))
            acc1 = accuracy_score(TST_L, predicted1)
            predicted2 = model2.predict(proj2(TST))
            acc2 = accuracy_score(TST_L, predicted2)
            predicted3 = model3.predict(proj3(TST))
            acc3 = accuracy_score(TST_L, predicted3)

            # Store accuracy for each K
            acclist_gd.append(acc1)
            acclist_eig.append(acc2)
            acclist_nepv.append(acc3)

        # Store accuracy for each p
        if p not in Acclist_gd:
            Acclist_gd[p] = []
        if p not in Acclist_eig:
            Acclist_eig[p] = []
        if p not in Acclist_nepv:
            Acclist_nepv[p] = []
        Acclist_gd[p].append(acclist_gd)
        Acclist_eig[p].append(acclist_eig)
        Acclist_nepv[p].append(acclist_nepv)

    # Store running time    
    Time_gd.append(time_gd)
    Time_eig.append(time_eig)
    Time_nepv.append(time_nepv)

avg_gd, avg_eig, avg_nepv = [], [], []
for p in [5, 10, 15, 20, 25]:
    avg_gd.append(np.mean(Acclist_gd[p], 0))
    avg_eig.append(np.mean(Acclist_eig[p], 0))
    avg_nepv.append(np.mean(Acclist_nepv[p], 0))